<a href="https://colab.research.google.com/github/iwatake2222/study_karaage_ai/blob/master/karaage_ai_03_01_markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
!unzip 2093_ruby_28087.zip

--2021-07-11 14:26:50--  https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip [following]
--2021-07-11 14:26:50--  https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 421747 (412K) [application/zip]
Saving to: ‘2093_ruby_28087.zip’

2093_ruby_28087.zip 100%[===================>] 411.86K  --.-KB/s    in 0.02s   

2021-07-11 14:26:51 (16.7 MB/s

In [ ]:
!pip install janome

     |████████████████████████████████| 19.7MB 6.3MB/s 


In [ ]:
import re

def normalize_text(text):
  text = re.sub(r"それ", "", text)
  text = re.sub(r"よう", "", text)
  text = re.sub(r"もの", "", text)
  text = re.sub(r"これ", "", text)
  text = re.sub(r"うち", "", text)
  text = re.sub(r"ここ", "", text)
  text = re.sub(r"こと", "", text)
  text = re.sub(r"もの", "", text)
  text = re.sub(r"そう", "", text)
  text = re.sub(r"\n", "", text)
  text = text.strip()
  return text

text_list = []
with open("dogura_magura.txt", encoding="shift_jis") as f:
  text_list = f.readlines()
new_text_list = []
for text in text_list:
  new_text_list.append(normalize_text(text))
text_list = new_text_list

In [ ]:
from janome.tokenizer import Tokenizer

def wakachigaki(text_list):
  t = Tokenizer()
  words = []
  for text in text_list:
    tokens = t.tokenize(text)
    for token in tokens:
      pos = token.part_of_speech.split(",")[0]
      # if pos == "名詞":
      words.append(token.surface)
  text =" ".join(words)
  return text

In [ ]:
word_list = [w for w in wakachigaki(text_list).split()]
word_list[0:10]

['ドグラ・マグラ',
 '夢野',
 '久作',
 '-------------------------------------------------------',
 '【',
 'テキスト',
 '中',
 'に',
 '現れる',
 '記号']

In [ ]:
test_text = ["私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。"]
test_text = wakachigaki(test_text)
test_text = test_text.replace("から あげ", "からあげ")
test_text = test_text.replace("お やつ", "おやつ")
test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。']

In [ ]:
def make_markov_model_1(word_list):
  markov = {}
  w1 = ""
  for word in word_list:
    if w1:
      if w1 not in markov:
        markov[w1] = []
      markov[w1].append(word)
    w1 = word
  return markov

In [ ]:
markov_model_test_1 = make_markov_model_1(test_word_list)

{'。': ['君', '私'],
 'おやつ': ['が'],
 'からあげ': ['が', 'を'],
 'が': ['好き', '好き'],
 'だ': ['。', '。'],
 'は': ['からあげ', 'からあげ', 'おやつ'],
 'を': ['食べる'],
 '君': ['は'],
 '好き': ['だ', 'だ'],
 '私': ['は', 'は'],
 '食べる': ['。']}

In [ ]:
def check_model(model, check_numb=10):
  count = 0
  for key in model.keys():
    if count >= 0:
      print("key:", key)
      print("value:", model[key])
      print("----------------")
    count += 1
    if count > check_numb:
      break
check_model(markov_model_test_1)

key: 私
value: ['は', 'は']
----------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
----------------
key: からあげ
value: ['が', 'を']
----------------
key: が
value: ['好き', '好き']
----------------
key: 好き
value: ['だ', 'だ']
----------------
key: だ
value: ['。', '。']
----------------
key: 。
value: ['君', '私']
----------------
key: 君
value: ['は']
----------------
key: を
value: ['食べる']
----------------
key: 食べる
value: ['。']
----------------
key: おやつ
value: ['が']
----------------


In [ ]:
def make_markov_model_2(word_list):
  markov = {}
  w1 = ""
  w2 = ""
  for word in word_list:
    if w1 and w2:
      if (w1, w2) not in markov:
        markov[(w1, w2)] = []
      markov[(w1, w2)].append(word)
    w1, w2 = w2, word
  return markov

In [ ]:
markov_model_2 = make_markov_model_2(word_list)
check_model(markov_model_2, 20)

key: ('ドグラ・マグラ', '夢野')
value: ['久作']
----------------
key: ('夢野', '久作')
value: ['-------------------------------------------------------', '全集']
----------------
key: ('久作', '-------------------------------------------------------')
value: ['【']
----------------
key: ('-------------------------------------------------------', '【')
value: ['テキスト']
----------------
key: ('【', 'テキスト')
value: ['中']
----------------
key: ('テキスト', '中')
value: ['に']
----------------
key: ('中', 'に')
value: ['現れる', 'ハッキリ', '、', '、', '走り込み', 'ヨボヨボ', '含ま', 'は', '納まっ', '凝固', 'ゴチャゴチャ', '落ち', '、', '追い込ん', '映っ', '突立', '、', '誘い込ん', '湧き', '落し', '並ん', 'は', '封じ込め', '、', '一つ', '、', 'は', '盛込ま', '、', '記述', '記述', '陳列', '含ま', '、', '潜伏', '引寄', 'タッタ', '含ま', 'は', 'は', '摘発', '、', '…', 'は', '落し', '胚胎', '尊', '。', 'は', 'も', '公', '並ぶ', 'は', '交', 'さま', '重なる', '。', '居る', 'トグロ', '立て', '立ち止まっ', '発見', '宿っ', '在る', '呼吸', '、', 'ホッ', '、', '、', '葬り', '立往生', '詰めかけ', 'は', '引っ', 'も', '平等', '往々', '、', '見知っ', 'は', '、', '潜在', '寝', '潜在', '包み込ん', '現われ

In [ ]:
import random

def generate_text_2(model, max_sentence):
  count_sentence = 0
  sentence = ""
  w1, w2 = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[w1, w2])
      sentence += tmp
      if tmp == "。":
        count_sentence += 1
        sentence += "\n"
      w1, w2 = w2, tmp
    except:
      w1, w2 = random.choice(list(model.keys()))
  return sentence

In [ ]:
print(generate_text_2(markov_model_2, 10))

の就寝後三十男を失恋自殺者、行路病者なぞの一切合財は、その原理から生れて来る額の禿《はなれ》で、父の御門《みかど》か賭博《ば》れてしまって、眼も当てられぬ患者もある位です。
現に当大学の名前を………のみならずＴ子の身代りとなっている私を憫《あわ》せ持って来いの第一頁ごとに赤《あたかという、玄宗皇帝というおやじ［＃「おやじ」に傍点］呉八代子の娘を遣るか』という精神病理学から観察した治療法というは、そんな事は知らぬか……なぞとは夢にも思いつきませんでした妾は、今話した「脳髄は物を供え、屍鬼をば、昨今の唯物文化の世界に浮いている無量無数に在って、声を上げた。
やがて明るい、美しい街筋に走り込んだ……又、思い出したな自己意識も、したな気持になり、汝空を翹《かけら》めいた。
しているから、私共も、現在の気持になっている細胞自身の現実の意識の中の硝子《ガラス》窓の外の霧を見ると又、実際のところＷとＭの二重人格の高かった斎藤博士の姿に似合わない、下らない習慣が、こんな不平を云わせたいのだろうとする心理遺伝によって演出されたでしたが、君を突き合わせて、この事件は大抵写して御座る事を察するを得ませぬが、にしてタッタ今地上に最初に映写し出された意識や、又は細い煙の中の二階に誘いをかけ、結跏趺座《けっかふざ》し男と戯れるところ……ブ――――――各種の遺跡によって推定するという事になった扉が。
日進月歩で開けて来る」と云うて、一層｜急《せ》の格言のあらわれではあるが…………よく御覧になりませぬので……いいかい。
フーム……ボ――ン……チャカポコチャカポコ……又は全然、性質は念のために、そのな表面的の知識が欠けているのじゃないかは知らず……したら、その頬の色がスーと消え失せてしまった。
第一箇条になったのである。
その強敵Ｗは『Ｗ』も、実際の進化した名誉慾だ。
すなわち若林君は………が併《しか》る瀬《せ》き声が、あの畠の縁に着陸致しますと。
数をつくして、眼科の田西博士と私の力が尽きた。



In [ ]:
def make_markov_model_5(word_list):
  markov = {}
  w1 = ""
  w2 = ""
  w3 = ""
  w4 = ""
  w5 = ""
  for word in word_list:
    if w1 and w2 and w3 and w4 and w5:
      if (w1, w2, w3, w4, w5) not in markov:
        markov[(w1, w2, w3, w4, w5)] = []
      markov[(w1, w2, w3, w4, w5)].append(word)
    w1, w2, w3, w4, w5 = w2, w3, w4, w5, word
  return markov

In [ ]:
import random

def generate_text_5(model, max_sentence):
  count_sentence = 0
  sentence = ""
  w1, w2, w3, w4, w5 = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[w1, w2, w3, w4, w5])
      sentence += tmp
      if tmp == "。":
        count_sentence += 1
        sentence += "\n"
      w1, w2, w3, w4, w5 = w2, w3, w4, w5, tmp
    except:
      # print("retry")
      w1, w2, w3, w4, w5 = random.choice(list(model.keys()))
  return sentence

In [ ]:
markov_model_5 = make_markov_model_5(word_list)
print(generate_text_5(markov_model_5, 10))

下になっている腕時計を見ますと、もう向うを向いて布団に潜っていて、櫛巻《くしま》きの頭だけしか見えませんでした。
――往来は眩《まぶ》しい緑色の中に、ツイ今しがた発見した黒い、留針《ピン》の頭ほどの焼け焦《こ》げ臭いにおいを嗅ぎ付けたに違いないのだ。
その子のＩに『お前が大学校を卒業する迄、私が無事でいたら、何もかも話して上げる』と云ったのは……イッタイいつの事だね……」呉一郎は、でも何かしら不安に鍬の上げ下げを凝視していたという。
しかもその時は既に半分呉青秀、半分は呉一郎の気持ちでいたと推定されているからね。
まして況《いわ》んや若林博士のな特殊な体質と頭脳を持った人間が、斯様《か》な不誠意を極めた屍体解剖を試みるのであろうか……と驚き迷い、呆れつつ若林博士の蒼白い顔と、額縁の中の斎藤博士の微笑とを交《かわ》る交る見比べた。
そんなにまで人格の高い立派な人が、何で変死なんかしたんだろうと疑いながら……。
しかし正木博士はビクともしない覚悟と方針とを考えまわしつつ……何ともいえず息苦しい、不可思議な昂奮に囚《とら》われた一つの驚くべき惨劇を演出しました。
してから見えます、あの三ツ並んだ土蔵《おくら》の方角を指さして見せながら、片手をオモヨさんの肩にかけて、二三度ゆすぶられますと、最前から火のに赤うなって身体《からだ》中がゾクゾクと寒気立《さむけだっ》て来たなので気がついて見ると、私の頭の中一パイにコダワっていた疑問だの、迷いだの、驚ろきだのいうが、みるみるにスースーと頭の中から蒸発して行った。
して北側の窓の処まで持って行って風呂敷をハタイた。
「……くり返して申しますが、その五代前の祖先、すなわちこの儀作の曾々祖父に当ります者は、福岡の御城下、鳥飼《とりかい》村に居りました名高い豪農で、同名｜儀十《ぎじゅう》と申す者で御座いました。

